In [95]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import pymssql
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import statsmodels.formula.api as smf
from datetime import datetime

### Process transactions file

In [96]:
FILE = 'TransactionHistory_3_23_2023.csv'

transact = pd.read_csv(FILE,parse_dates=['CreateDate'])
transact = transact.rename(columns={"Symbol": "ticker", "CreateDate": "date"})
transact['Price'] = transact.Price.apply(lambda x: x[1:]).astype(float)
transact['Quantity'] = transact.Quantity.astype(int)
transact['date'] = transact.date.dt.date.astype(str)
transact = transact.sort_values(by=["ticker", "date",'TransactionType'])
transact = transact.drop_duplicates(subset=['ticker','Quantity','Price','date'],keep='first')  # because short proceeds records show up twice (I think)
transact.Amount = transact.Amount.map(
    lambda x: float(x[1:].replace(",", "")) if x[0] != "(" else -float(x[2:-1].replace(",", ""))
)    

cashflow = transact.groupby("date").Amount.sum()
cash = 1000000 + cashflow.cumsum()

trades = transact[transact.TransactionType != 'Dividends']
trades = trades.groupby(["ticker", "date"]).Quantity.sum()

positions = trades.groupby("ticker").cumsum()       # sum up trades over time to get positions

### Read closing prices from Yahoo

In [97]:
prices = yf.download(list(transact.ticker.unique()), start="2022-11-04")

prices = prices.stack()
prices.index.names = ["date", "ticker"]
prices = prices.reset_index()[["date", "ticker", "Close"]]
prices.date = prices.date.dt.date.astype(str)
prices = prices.set_index(["ticker", "date"])

[*********************100%***********************]  96 of 96 completed

2 Failed downloads:
- FFHL: No data found for this date range, symbol may be delisted
- SPNE: No data found for this date range, symbol may be delisted


### Combine transactions with prices

In [98]:
df = prices.merge(positions, left_index=True, right_index=True, how="left")
indx = pd.MultiIndex.from_product(
    (df.index.unique("ticker"), df.index.unique("date"))
)
df = df.reindex(indx)                                       # expand dataframe to all (ticker, date) pairs

df["Price"] = transact.set_index(["ticker", "date"]).Price  # use transaction prices when available
df["Price"] = df.Price.fillna(df.Close)                     # use closing prices on non-transaction dates
df["Price"] = df.groupby("ticker").Price.ffill()            # filling forward for delisted stocks

df["Quantity"] = df.groupby("ticker").Quantity.ffill()      # fill quantities until next transaction date
df["Value"] = df.Quantity * df.Price

### Compute portfolio value and daily returns

In [99]:
portfolio = df.groupby("date").Value.sum()
portfolio = pd.DataFrame(portfolio).merge(cash, on="date", how="left")
portfolio.columns = ["stocks", "cash"]
portfolio["cash"] = portfolio.cash.ffill()
equity = portfolio.stocks + portfolio.cash
ret = equity.pct_change()

### Sharpe ratio, etc.

In [100]:
print(f"annualized average return = {252*ret.mean(): .1%}")
print(f"annualized std dev is {np.sqrt(252)*ret.std(): .1%}")
print(f"annualized Sharpe ratio is {np.sqrt(252)*ret.mean()/ret.std(): .1%}")

annualized average return =  43.3%
annualized std dev is  55.8%
annualized Sharpe ratio is  77.6%


### Plot cumulative return

In [101]:
cum_ret = equity/1000000
trace1 = go.Scatter(
    x = cum_ret.index.to_list(),
    y = cum_ret-1,
    mode="lines",
    hovertemplate="%{x}<br>cumulative ret = %{y:.2%}<extra></extra>"
)

fig = go.Figure(trace1)

fig.update_yaxes(
    title_text="<b>Cumulative Return</b>", 
    tickformat=".0%", 
)

fig.update_layout(
    template="plotly_white",
    font_size=16,
    showlegend=False,
)

fig.write_image("cqa_figures/cumret.jpeg")
fig.show()

### Plot drawdowns

In [102]:


cum_max = cum_ret.expanding().max()
drawdown = cum_ret/cum_max - 1

trace2 = go.Scatter(
    x = drawdown.index.to_list(),
    y = drawdown,
    mode="lines",
   hovertemplate="%{x}<br>drawdown = %{y:.2%}<extra></extra>"
)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1, secondary_y=False)
fig.add_trace(trace2, secondary_y=True)

plotly_template = pio.templates["plotly_white"]
colors = plotly_template.layout.colorway

fig.update_yaxes(
    title_text="<b>Cumulative Return</b>", 
    title_font=dict(color=colors[0]), 
    color=colors[0], 
    tickformat=".0%", 
    secondary_y=False
)
fig.update_yaxes(
    title_text="<b>Drawdown</b>", 
    title_font=dict(color=colors[1]), 
    color=colors[1], 
    tickformat=".0%", 
    secondary_y=True
)

fig.update_layout(
    template="plotly_white",
    font_size=16,
    showlegend=False,
)

fig.write_image("cqa_figures/drawdown.jpeg")
fig.show()

### Compare to S&P 500 (SPY)

In [103]:
spy = yf.download("SPY", start="2022-11-03")["Adj Close"].pct_change().dropna()
spy.index = [datetime.strftime(x, "%Y-%m-%d") for x in spy.index]

rets = pd.concat((ret, spy), axis=1)
rets.columns = ["ret", "spy"]

result = smf.ols("ret~spy", rets).fit()
alpha = result.params[0]
beta = result.params[1]
info_ratio = alpha / np.sqrt(result.mse_resid)

spy = beta*spy
active = ret - spy

trace1 = go.Scatter(
    x = ret.index.to_list(),
    y = (1+ret).cumprod()-1,
    name = "Total return",
    hovertemplate="%{x}<br>total=%{y:.2%}<extra></extra>"
)

trace2 = go.Scatter(
    x = active.index.to_list(),
    y = (1+active).cumprod()-1,
    name = "Active part of return",
    hovertemplate="%{x}<br>active=%{y:.2%}<extra></extra>"
)
trace3 = go.Scatter(
    x = spy.index.to_list(),
    y = (1+spy).cumprod()-1,
    name = "S&P 500 part of return",
    hovertemplate="%{x}<br>spy=%{y:.2%}<extra></extra>"
)
fig = go.Figure()
for trace in [trace1, trace3, trace2]:
    fig.add_trace(trace)

fig.update_layout(
    yaxis_title="Cumulative Return",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickformat=".0%",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)
fig.write_image("cqa_figures/spy.jpeg")
fig.show()

[*********************100%***********************]  1 of 1 completed


In [104]:
print(f"annualized alpha with respect to SPY = {252*result.params[0]:.1%}")
print(f"beta with respect to SPY = {result.params[1]:.2f}")
print(f"annualized info ratio with respect to SPY = {np.sqrt(252)* alpha / np.sqrt(result.mse_resid):.1%}")


annualized alpha with respect to SPY = 38.5%
beta with respect to SPY = 0.30
annualized info ratio with respect to SPY = 69.0%


### Compare to Russell 2000 (IWM)

In [105]:
spy = yf.download("IWM", start="2022-11-03")["Adj Close"].pct_change().dropna()
spy.index = [datetime.strftime(x, "%Y-%m-%d") for x in spy.index]

rets = pd.concat((ret, spy), axis=1)
rets.columns = ["ret", "spy"]

result = smf.ols("ret~spy", rets).fit()
alpha = result.params[0]
beta = result.params[1]
info_ratio = alpha / np.sqrt(result.mse_resid)

spy = beta*spy
active = ret - spy

trace1 = go.Scatter(
    x = ret.index.to_list(),
    y = (1+ret).cumprod()-1,
    name = "Total return",
    hovertemplate="%{x}<br>total=%{y:.2%}<extra></extra>"
)

trace2 = go.Scatter(
    x = active.index.to_list(),
    y = (1+active).cumprod()-1,
    name = "Active part of return",
    hovertemplate="%{x}<br>active=%{y:.2%}<extra></extra>"
)
trace3 = go.Scatter(
    x = spy.index.to_list(),
    y = (1+spy).cumprod()-1,
    name = "Russell 2000 part of return",
    hovertemplate="%{x}<br>iwm=%{y:.2%}<extra></extra>"
)
fig = go.Figure()
for trace in [trace1, trace3, trace2]:
    fig.add_trace(trace)

fig.update_layout(
    yaxis_title="Cumulative Return",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickformat=".0%",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)
fig.write_image("cqa_figures/russell.jpeg")
fig.show()

[*********************100%***********************]  1 of 1 completed


In [106]:
print(f"annualized alpha with respect to IWM = {252*result.params[0]:.1%}")
print(f"beta with respect to IWM = {result.params[1]:.2f}")
print(f"annualized info ratio with respect to IWM = {np.sqrt(252)* alpha / np.sqrt(result.mse_resid):.1%}")

annualized alpha with respect to IWM = 45.1%
beta with respect to IWM = 0.26
annualized info ratio with respect to IWM = 81.0%


### Get industries and some stock characteristics

In [60]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 

conn = create_engine(string).connect()

In [94]:
today = pd.read_sql("select ticker, famaindustry, agr, bm, marketcap, operprof, mom12m, retvol from today", conn)
complete = df.reset_index().merge(today, on="ticker", how="outer")

oneday = complete[complete.date=="2023-02-01"]
longs = oneday[oneday.Quantity>0]
shorts = oneday[oneday.Quantity<0]
shorts["Value"] = - shorts.Value


KeyError: 'ticker'

### Long industries

In [51]:
fig = px.pie(longs, values="Value", names="famaindustry")
fig.write_image("cqa_figures/long_industries.jpeg")
fig.show()

### Short industries

In [16]:
fig = px.pie(shorts, values="Value", names="famaindustry")
fig.write_image("cqa_figures/short_industries.jpeg")
fig.show()

,ticker,Quantity,Close,Price,Value,siccd,date,roeq,roaq,acc,...,retvol,name,exchange,sicsector,sicindustry,famaindustry,sector,industry,scalemarketcap,scalerevenue


### Net exposures: size and characteristics

In [26]:
for char in "agr bm marketcap operprof mom12m retvol".split():
    df[char+"_group"] = pd.qcut(df[char], 5, labels=range(1, 6))

agr = df.dropna(subset=["marketcap", "agr", "Value"]).groupby(["marketcap", "agr"]).Value.sum().unstack()
bm = df.dropna(subset=["marketcap", "bm", "Value"]).groupby(["marketcap", "bm"]).Value.sum().unstack()
operprof = df.dropna(subset=["marketcap", "operprof", "Value"]).groupby(["marketcap", "operprof"]).Value.sum().unstack()
mom12m = df.dropna(subset=["marketcap", "mom12m", "Value"]).groupby(["marketcap", "mom12m"]).Value.sum().unstack()
retvol = df.dropna(subset=["marketcap", "retvol", "Value"]).groupby(["marketcap", "retvol"]).Value.sum().unstack()

,ticker,date,price,closeadj,industry,ret
0,A,2022-11-01,140.89,140.678,Measuring and Control Equipment,NaN
1,A,2022-11-02,135.27,135.067,Measuring and Control Equipment,-0.039885
2,A,2022-11-03,134.46,134.258,Measuring and Control Equipment,-0.005990
3,A,2022-11-04,136.08,135.876,Measuring and Control Equipment,0.012051
4,A,2022-11-07,138.03,137.823,Measuring and Control Equipment,0.014329


In [ ]:
chars = "agr bm operprof mom12m retvol".split()
data = [agr, bm, operprof, mom12m, retvol]

for char, d in zip(chars, data):
    trace = go.Heatmap(
        x=d.columns.to_list(),
        y=d.index.to_list(),
        z=d ,
        colorscale='Viridis',
        texttemplate="%{z:.1%}"
    )
    fig = go.Figure(trace)
    fig.update_layout(
        template="plotly_white",
        font_size=16
)
fig.write_image(f"cqa_figures/{char}.jpeg")
fig.show()